In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import tensorflow as tf
%matplotlib inline
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten 
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import BatchNormalization
from keras import optimizers
from keras import callbacks
from keras.utils import to_categorical
from keras.layers import Activation
import cv2
from keras.layers import UpSampling2D
from albumentations import PadIfNeeded
from albumentations import HorizontalFlip
from albumentations import VerticalFlip   
from albumentations import CenterCrop   
from albumentations import Crop
from albumentations import Compose
from albumentations import Transpose
from albumentations import RandomRotate90
from albumentations import Rotate
from albumentations import RandomSizedCrop
from albumentations import OneOf
from albumentations import CLAHE
from albumentations import RandomBrightnessContrast    
from albumentations import RandomGamma 
from keras.applications import vgg19
from keras import layers
from keras import models
import keras.backend as K
from keras.applications.vgg19 import VGG19
from IPython.display import Image, display
from keras.models import Model
from keras.callbacks import History 
history = History()
from pathlib import Path
import keras
tf.compat.v1.disable_eager_execution()

 ## Extract and analyse files 

In [ ]:
'''
For given set of images, map the DRR's to masks
'''
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
path = Path("../input/digitally-reconstructed-radiographs-drr-bones")
drr = list(map(str, list(path.glob("**/*.png"))))
bone_drr = pd.DataFrame([(x, x.replace('.png','_mask.png')) for x in drr if not x.endswith('_mask.png')])
bone_drr.columns = ['image','bones']
bone_drr.head(10)

## Analyse image and their masks

In [ ]:
img = cv2.imread(bone_drr['image'][0])
plt.imshow(img)

In [ ]:
img1 = cv2.imread(bone_drr['bones'][0])
plt.imshow(img1)

In [ ]:
img = cv2.imread(bone_drr['image'][1])
plt.imshow(img)

In [ ]:
img = cv2.imread(bone_drr['bones'][1])
plt.imshow(img)

## Test train split

In [ ]:
seed = 1000
np.random.seed(seed)
tf.random.set_seed(seed)
bone_drr = bone_drr.sample(frac=1, random_state=seed)
bone_drr_train = bone_drr[:160].reset_index(drop=True)
bone_drr_test = bone_drr[160:].reset_index(drop=True)

In [ ]:
bone_drr_train.head(5)

In [ ]:
bone_drr_test.head(5)

In [ ]:
x_train = []
y_train = []
x_test  = []
y_test  = []
for i in range(len(bone_drr_train)):
    img = cv2.imread(bone_drr_train['image'][i] , 0)
    img = cv2.resize(img,(512,512))
    img = img.astype(np.float32)
    img-=img.mean()
    img/=img.std()
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    x_train.append(img)
    mask = cv2.imread(bone_drr_test['bones'][1])
    mask = mask/255
    mask = cv2.resize(mask,(512,512))
    y_train.append(mask)
for i in range(len(bone_drr_test)):
    img = cv2.imread(bone_drr_test['image'][i] , 0)
    img = cv2.resize(img,(512,512))
    img = img.astype(np.float32)
    img-=img.mean()
    img/=img.std()
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    mask = cv2.imread(bone_drr_test['bones'][1])
    mask = mask/255
    mask = cv2.resize(mask,(512,512))
    x_test.append(img)
    y_test.append(mask)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test  = np.array(x_test)
y_test  = np.array(y_test)

In [ ]:
np.shape(x_train)

In [ ]:
plt.imshow(x_train[100])

In [ ]:
plt.imshow(y_train[100])

## Build UNET Model

Specifications of UNET dilations: https://arxiv.org/pdf/2003.10839.pdf

In [ ]:
img_shape = (512,512,3)
input_img = layers.Input(img_shape)
l11 = layers.Conv2D(44,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(input_img)
l11 = layers.Conv2D(44,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(input_img)
l13 = layers.MaxPooling2D(strides=(2,2) , pool_size=((2,2)))(l11)

l21 = layers.Conv2D(88,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(l13)
l21 = layers.Conv2D(88,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(l13)
l23 = layers.MaxPooling2D(strides=(2,2) , pool_size=((2,2)))(l21)

l31 = layers.Conv2D(176,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(l23)
l31 = layers.Conv2D(176,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(l31)
l33 = layers.MaxPooling2D(strides=(2,2) , pool_size=((2,2)))(l31)

d1 = layers.Conv2D(176,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(l33)
d2 = layers.Conv2D(176,kernel_size=3,dilation_rate=2,kernel_initializer="he_normal",padding="same",activation="relu")(l33)
d3 = layers.add([d1 , d2])
u1 = layers.UpSampling2D((2,2))(d3)

u1 = layers.Conv2D(88,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u1)
u1 = layers.concatenate([l31 , u1])
u1 = layers.Conv2D(88,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u1)
u1 = layers.Conv2D(88,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u1)

u2 = layers.UpSampling2D((2, 2))(u1)
u2 = layers.Conv2D(44,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u2)
u2 = layers.concatenate([l21 , u2])
u2 = layers.Conv2D(44,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u2)
u2 = layers.Conv2D(44,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u2)

u3 = layers.UpSampling2D((2, 2))(u2)
u3 = layers.Conv2D(22,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u3)
u3 = layers.concatenate([l11 , u3])
u3 = layers.Conv2D(22,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u3)
u3 = layers.Conv2D(22,kernel_size=3,dilation_rate=1,kernel_initializer="he_normal",padding="same",activation="relu")(u3)

output_layer = layers.Conv2D(1,1,activation="tanh")(u3)
model = Model(inputs = input_img , outputs = output_layer)


In [ ]:
model

In [ ]:
model.summary()

## Calculate Loss using VGG-19

In [ ]:
img = keras.utils.model_to_dot(
    model,
    show_shapes=True,
    expand_nested=True
)
pdot = Image(img.create_png())
display(pdot)

In [ ]:
vgg19_model = VGG19(include_top=False, weights="imagenet", input_shape=(512,512, 3))
loss_model = models.Model(inputs=vgg19_model.input , outputs=vgg19_model.get_layer("block3_conv3").output)
loss_model.trainable = False
def vgg_loss(y_true, y_pred):
    y_pred = tf.image.grayscale_to_rgb(y_pred,name=None)
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))
loss_model.summary()

In [ ]:
img = keras.utils.model_to_dot(
    loss_model,
    show_shapes=True,
    expand_nested=True
)
pdot = Image(img.create_png())
display(pdot)

## Train the model

In [ ]:
model.compile(optimizer="adam", loss=vgg_loss , metrics=[vgg_loss])

In [ ]:
datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1.0,
    zoom_range=0.2,
    fill_mode="nearest",
    cval=0,
)
test_gen = ImageDataGenerator(rescale=1.0)
batch_size = 4
tsteps_per_epoch = np.shape(x_train)[0] + batch_size - 1
vsteps_per_epoch = np.shape(x_test)[0] + batch_size - 1
epoch_info = model.fit(x_train,y_train, steps_per_epoch=tsteps_per_epoch ,epochs=100, validation_data=test_gen.flow(x_test,y_test,batch_size=batch_size),validation_steps=vsteps_per_epoch , callbacks=[history])

In [ ]:
print(epoch_info.info)

## Test the model

In [ ]:
img = x_test[1]
pred = model.predict(img)
pred = model.resize(pred , (512,512))
plt.imshow(pred)